In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import os
from getpass import getpass
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
import time
from pymongo import MongoClient

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["ironhack"]
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [4]:
db.list_collection_names()

['nyneigh', 'nyrest', 'companies', 'restaurants']

In [5]:
c = db.get_collection("companies")

In [6]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [7]:
category = c.distinct("category_code")
print(category)

[None, 'advertising', 'analytics', 'automotive', 'biotech', 'cleantech', 'consulting', 'design', 'ecommerce', 'education', 'enterprise', 'fashion', 'finance', 'games_video', 'government', 'hardware', 'health', 'hospitality', 'legal', 'local', 'manufacturing', 'medical', 'messaging', 'mobile', 'music', 'nanotech', 'network_hosting', 'news', 'nonprofit', 'other', 'photo_video', 'public_relations', 'real_estate', 'search', 'security', 'semiconductor', 'social', 'software', 'sports', 'transportation', 'travel', 'web']


In [8]:
filter = {"category_code":{"$regex": "ecommerce|games_video|hardware|analytics|design|mobile|software|web"}}
filter_2 = {"offices.country_code": "DEU"}
projection = {"name": 1, "_id": 0, "category_code":1, "total_money_raised": 1, "offices.city":1, "offices.country_code": 1}
comp= list(c.find({
    '$and':
    [filter, filter_2]},projection).sort("total_money_raised", -1))
comp

[{'name': 'Karo Internet',
  'category_code': 'games_video',
  'total_money_raised': '€850k',
  'offices': [{'city': 'Cologne', 'country_code': 'DEU'}]},
 {'name': 'kinkon',
  'category_code': 'web',
  'total_money_raised': '€750k',
  'offices': [{'city': 'Kaiserslautern', 'country_code': 'DEU'}]},
 {'name': 'Moozey',
  'category_code': 'mobile',
  'total_money_raised': '€70k',
  'offices': [{'city': 'Munich', 'country_code': 'DEU'}]},
 {'name': 'Moozey',
  'category_code': 'mobile',
  'total_money_raised': '€70k',
  'offices': [{'city': 'Munich', 'country_code': 'DEU'}]},
 {'name': 'RODECO ICT Services',
  'category_code': 'web',
  'total_money_raised': '€600k',
  'offices': [{'city': 'Boskoop', 'country_code': 'NLD'},
   {'city': 'Frankfurt am Main', 'country_code': 'DEU'},
   {'city': 'Bucharest', 'country_code': 'ROM'}]},
 {'name': 'zweitgeist',
  'category_code': 'web',
  'total_money_raised': '€500k',
  'offices': [{'city': 'Hamburg', 'country_code': 'DEU'}]},
 {'name': 'Transins

In [9]:
#offices_ = comp[0]["offices"][0]["city"]
#offices_

In [10]:
cities = [] 
for i in range(len(comp)):
    for j in range(len(comp[i]["offices"])):
        cities.append(comp[i]["offices"][j]["city"])

In [11]:
country = [] 
for i in range(len(comp)):
    for j in range(len(comp[i]["offices"])):
        country.append(comp[i]["offices"][j]["country_code"])

In [25]:
df = pd.DataFrame(comp)
df.drop(columns=["offices"],inplace=True)

In [43]:
df2

,city,country,tmp
0,Cologne,DEU,1
1,Kaiserslautern,DEU,1
2,Munich,DEU,1
3,Munich,DEU,1
4,Boskoop,NLD,1
...,...,...,...
513,"Cheltenham, Glos",GBR,1
514,Mannheim,DEU,1
515,berlin,DEU,1
516,Berlin,DEU,1


In [46]:
df2["city"].value_counts()[:20]

Berlin               61
Munich               32
Hamburg              24
London               19
                     14
San Francisco        12
Paris                11
New York             10
Madrid                8
Stuttgart             8
Cologne               8
Tokyo                 6
DÃ¼sseldorf           6
Shanghai              6
Frankfurt             5
Beijing               4
Frankfurt am Main     4
Karlsruhe             4
Singapore             4
Boston                4
Name: city, dtype: int64

In [49]:
square_berlin = 891,8
square_hamburg = 755,2
square_munich = 310,7
num_comp_berlin = 61
num_comp_munich = 32
num_comp_hamburg = 24

TypeError: unsupported operand type(s) for /: 'int' and 'tuple'